# Here is a title
here is some text

In [7]:
#%%capture 
#hide output of this cell

import tensorflow as tf
print(tf.__version__)


# Qua fatti una seconda var con il tuo path, cosi a seconda di chi lo sta usando mettiamo un hastag allinizio
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/--.tfrecord'


2.10.0


First of all, inspect the raw TFR file structure to get an idea of how its built and it looks. 

In [24]:
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Iterate through the dataset and print the raw example data
for record in dataset:
    n = 1
    example = tf.train.Example.FromString(record.numpy())
    print(example, n)
    n +=1

features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 399
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--cB2ZVjpnA"
      }
    }
  }
}
 1
features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 0
        value: 451
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--PJHxphWEs"
      }
    }
  }
}
 1
features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value

In [22]:
# Qua faccio un check per capire il data type del feature labels, perché piú avanti mi crea problemi
# Define a parsing function to extract and decode the data
## Boh sto qua semplicemente restituisce il data type specificato su tf.io.VarLenFeature(tf.int64), 
## e non il data type reale. Anche cambiare la parte 'features/feature/video_id' non ha nessun effetto


def parse_function(example_proto):
    feature_description = {
        'features/feature/video_id': tf.io.VarLenFeature(tf.int64),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Get the 'labels' feature
    labels = example['features/feature/video_id']
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset
parsed_data = dataset.map(parse_function)

# Print the data type of the 'labels' feature
for labels in parsed_data:
    dtype = labels.values.dtype
    print(f'Data Type of "labels" Feature: {dtype}')

Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>


In [57]:
# Nuovo approcio da stack overflow: https://stackoverflow.com/questions/65007191/how-to-read-decode-tfrecords-file-see-the-images-inside-and-do-augmentation
# questo funyiona un pó meglio, si riescono ad accedere le singole entry. Sembra che alla fine file 
# TFRecord siano dizionari di dizionari di dizionari o qualcosa del genere. 
dataset = tf.data.TFRecordDataset(tfrecord_path)

for i, raw_record in enumerate(dataset.take(3)): # keeping at a minimal example of 3, remove for act code
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    info = {}
    for k, v in example.features.feature.items():
        if k == 'labels':
            print(k, v, 'NEXT')
            info[k] = v




print('INFO: \n', info)




labels int64_list {
  value: 399
}
 NEXT
labels int64_list {
  value: 0
  value: 451
}
 NEXT
labels int64_list {
  value: 27
  value: 466
}
 NEXT
INFO: 
 {'labels': int64_list {
  value: 27
  value: 466
}
}


Open and explore TFRecord files. 
Goal is to filter them in order to only keep files containing a certain label, like thunder

In [20]:
# Qua poi in teoria volgio leggere i label (e solo quelli) in modo che piú avanti si possano filtrare i file che contengono il label thunder (287,/m/0ngt1)
# La lista dei vari label si puó trovare nel file csv che ho caricato

# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'features/feature/': tf.io.VarLenFeature(tf.float32),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the 'labels' feature
    labels = tf.sparse.to_dense(example['features/feature/'])
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for labels in dataset.map(parse_function):
    # Print the 'labels' feature
    print(f'time: {labels.numpy()}')

time: []
time: []
time: []
time: []
time: []
time: []
time: []


In [11]:
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/-0.tfrecord'

# tfrecord_dataset = tf.data.TFRecordDataset(tfrecord_path)


# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'context/feature/video_id': tf.io.FixedLenFeature([], tf.string),
        'context/feature/start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/labels': tf.io.VarLenFeature(tf.int64),
        'feature_lists/feature_list/audio_embedding': tf.io.VarLenFeature(tf.string),
        # Define other features and their types based on your TFRecord structure
    }
    
    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # If 'end_time_seconds' may not be present in all records, use allow_missing=True
    if 'context/feature/end_time_seconds' in example:
        end_time_seconds = example['context/feature/end_time_seconds']
    else:
        end_time_seconds = 0.0  # Provide a default value
    
    
    # Decode the VarLenFeatures
    example['context/feature/labels'] = tf.sparse.to_dense(example['context/feature/labels'], default_value=0)
    example['feature_lists/feature_list/audio_embedding'] = tf.sparse.to_dense(example['feature_lists/feature_list/audio_embedding'], default_value='b')
    
    return example

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for record in dataset.map(parse_function):
    video_id = tf.strings.decode(record['context/feature/video_id'], 'utf-8')
    start_time_seconds = tf.make_ndarray(record['context/feature/start_time_seconds'])[0]
    end_time_seconds = tf.make_ndarray(record['context/feature/end_time_seconds'])[0]
    labels = tf.make_ndarray(record['context/feature/labels'])
    audio_embedding = record['feature_lists/feature_list/audio_embedding']
    
    print(f'Video ID: {video_id}')
    print(f'Start Time (seconds): {start_time_seconds}')
    print(f'End Time (seconds): {end_time_seconds}')
    print(f'Labels: {labels}')
    print(f'Audio Embedding: {audio_embedding}')



InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: context/feature/end_time_seconds (data type: float) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]